In [ ]:
from qwen_vl_utils import process_vision_info
from datasets import load_dataset, concatenate_datasets
from PIL import Image
from tqdm import tqdm
import numpy as np
from datetime import datetime

import json

def parse_json(json_output):
    pattern = r"\[([0-9\.]+(?:, ?[0-9\.]+)*)\]"
    matches = re.findall(pattern, json_output)
    coordinates = [
        [float(num) if "." in num else int(num) for num in match.split(",")]
        for match in matches
    ]
    return coordinates

with open("/home/pengziyang/MultimodalReasoningCompetition/Task1/datasets/VG_RS/ovdino_label/0804_ovdino_candidate_box_iou30.jsonl", 'r', encoding='utf-8') as f:
    anno_ovdino = [json.loads(line) for line in f]
    
with open("/home/pengziyang/MultimodalReasoningCompetition/Task1/datasets/VG_RS/ovdino_label/0805_llmdet_candidate.jsonl", 'r', encoding='utf-8') as f:
    anno_llmdet = [json.loads(line) for line in f]
    
for llmdet, ovdino in tqdm(zip(anno_llmdet, anno_ovdino)):
    boxes_llmdet = parse_json(llmdet['problem'])
    boxes_ovdino = parse_json(ovdino['problem'])
    box_all = []
    box_all.extend(boxes_llmdet)
    box_all.extend(boxes_ovdino)
    ovdino['problem'] = ovdino['problem'].replace(str(boxes_ovdino), str(box_all))
    
    
with open("/home/pengziyang/MultimodalReasoningCompetition/Task1/datasets/VG_RS/ovdino_label/0805_candidate_ovdino_llmdet.jsonl", 'w', encoding='utf-8') as f:
    for anno in anno_ovdino:
        json.dump(anno, f, ensure_ascii=False)
        f.write("\n")